#### Compose Two Different Static Maze2D Models

In [ ]:
import numpy as np
import os, sys, torch
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
sys.path.append('.') ## please add the project root dir accordingly
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cudnn.deterministic = True
import diffuser.utils as utils
from diffuser.models import GaussianDiffusionPB
from diffuser.guides.policies_compose import PolicyCompose
from datetime import datetime
import os.path as osp
from tqdm import tqdm
from diffuser.utils.comp.comp_setup import ComposedParser
from diffuser.utils.jupyter_utils import suppress_stdout
from tap import Tap
import numpy as np
np.set_printoptions(precision=3)

#---------------------------------- setup ----------------------------------#

## You need to download 'rm2d-comp' from OneDrive Link in README.md to launch
config = "config/comp/Comp_rSmaze_nw6nw3_hExt0507_engy_add.py"
class Parser(Tap):
    config: str = config
    plan_n_maze: int = 0 # placeholder

from diffuser.guides.comp_plan_env import ComposedEnvPlanner
from diffuser.guides.comp_plan_helper import ComposedDiffusionPlanner

sys.argv = ['PlaceHolder.py', '--config', config]

with suppress_stdout():
    args_comp = Parser().parse_args()
    cp = ComposedParser(args_comp.config)
    args_train_list, args_list = cp.setup_args_list(args_comp,)
    dplanner = ComposedDiffusionPlanner(args_train_list, args_list, cp.comp_dataset)

plan_env_list = dplanner.plan_env_list


#### Create an Env

In [ ]:
from diffuser.utils.rm2d_render import RandStaticMazeRenderer
import imageio
# import pb_diff_envs.environment.static.rand_maze2d_renderer as rmr; reload(rmr)
from pb_diff_envs.environment.static.rand_maze2d_renderer import RandMaze2DRenderer
rootdir = './visualization/'

bs = 50
env_source = 'dataset' # or 'custom'
env_source = 'custom'

if env_source == 'custom':
    env_id = 99 # placeholder
    wall_locations_c = [
        np.array([[1.13 , 3.184],
            [1.571, 1.849],
            [2.857, 4.093],
            [3.4  , 2.394],
            [3.563, 1.143],
            [4.333, 3.949],]),
        np.array([
            [1.069, 1.257],
            [3.101, 2.45],
            [2.714, 4.14 ]]),
        ]
    
    ## size of the obstacles 1x1 square block
    if len(wall_locations_c) == 2:
        hExt_list_c = [np.array([[0.5, 0.5],]*6), np.array([[0.7, 0.7],] * len(wall_locations_c[1]) )]
    else:
        hExt_list_c = [np.array([[0.5, 0.5],]*6), ]
    env = plan_env_list.create_env_by_pos(env_id, wall_locations_c, hExt_list_c)
    
elif env_source == 'dataset':
    env_id = 30 # input an index
    env = plan_env_list.create_single_env(env_id)


n_walls_1 = len(env.wloc_list_c[0]) ## num of the smaller 1x1 blocks
n_walls_2 = len(env.wloc_list_c[1]) ## num of the larger 1.4x1.4 blocks

obs_start_np = np.array( [ [ 1.20, 0.2 ] ], dtype=np.float32 ).repeat( bs, 0 )
target_np = np.array( [ [3.89, 4.55] ], dtype=np.float32  ).repeat( bs, 0 )


e_wlocs = env.wall_locations
env_hExts = env.wall_hExts # (num_walls, 2)
print(f'[Training Time] model 1 only trained on 3 obstacles (o); model 2 # of obstacles: 6')
print(f'[Now] # of walls: {len(e_wlocs)}')

mz_renderer = RandMaze2DRenderer(num_walls_c = list(map(lambda x: len(env.wloc_list_c[x]), list(range(len(env.wloc_list_c))) )), fig_dpi=200)
mz_renderer.up_right = 'empty'
mz_renderer.config['no_draw_traj_line'] = True
mz_renderer.config['no_draw_col_info'] = True

img = mz_renderer.renders( np.concatenate([obs_start_np[0:1,], target_np[0:1,],]), plan_env_list.maze_size, e_wlocs, env_hExts )
utils.plt_img(img, dpi=80)


In [ ]:
print( env.wall_locations.shape )
print( env.wall_locations )

In [ ]:
from importlib import reload
import imageio
import diffuser.guides.comp_plan_env as dcpe; reload(dcpe)
from diffuser.guides.comp_plan_env import ComposedEnvPlanner, ComposedRM2DReplanner

plan_env_list = dplanner.plan_env_list
rm2d_planner = ComposedEnvPlanner()

policy = dplanner.policy
policy.ddim_eta = 1.0
print(f' policy: {policy.cg_w}')
use_ddim = True
repl_dn_steps = 5

wloc_np = env.wall_locations ## important
policy.comp_type = 'share-model2fix'
# n_walls_1 is the smaller block
policy.num_walls_c = torch.tensor([ min(6, n_walls_1+n_walls_2), n_walls_2 ])
print(f'policy.num_walls_c: {policy.num_walls_c}')

repl_config = dict(use_ddim=use_ddim, dn_steps=repl_dn_steps)
replanner = ComposedRM2DReplanner(policy, dplanner.train_normalizer_list[0], repl_config, 'cuda')


obs_start_arr = utils.to_torch(obs_start_np,  device='cpu')
target_arr = utils.to_torch( target_np, device='cpu' )
wloc_input = wloc_np[None, ].repeat(bs, axis=0) # (10, 2) -> (bs, 10, 2)
wloc_input = wloc_input.reshape(bs, -1)
wall_locations = utils.to_torch( wloc_input,  device='cpu' )


use_normed_wallLoc = False
plan_env_config = utils.Config(None,
    env_mazelist=dplanner.plan_env_list,
    prob_permaze=1,
    samples_perprob=bs,
    obs_selected_dim=(0,1),
    horizon_pl_list=[48, 52, 56, 60, 64],
    horizon_wtrajs=48,
    do_replan=False,
    replanner=replanner,
)
with suppress_stdout():
    vis_modelout_path_list, samples_mulho, _ = rm2d_planner.plan_env_interact(
        env, policy, obs_start_arr, target_arr, wall_locations, use_normed_wallLoc, plan_env_config,)



In [ ]:
from diffuser.utils.jupyter_utils import get_all_suc_trajs

all_candidates = []
for trajs in samples_mulho:
    all_candidates.extend(trajs.observations)
## pick out successful trajectories
print('len all_candidates:', len(all_candidates))
env.robot.collision_eps = 1e-2
suc_trajs, suc_idxs = get_all_suc_trajs(env, trajs=all_candidates, goal=target_np)

In [ ]:
## Rendering
from diffuser.utils.rm2d_render import RandStaticMazeRenderer
rd = RandStaticMazeRenderer(plan_env_list)

tmp_path = f'{rootdir}/luotest.png'
renderer = plan_env_list.model_list[env_id].renderer
renderer.fig_dpi = 40 # lower resolution for fast render
renderer.up_right = 'default'
renderer.config['no_draw_traj_line'] = False
renderer.config['no_draw_col_info'] = False

# rd.composite( tmp_path, unnm_traj[:10],  np.array([env_id])) # visualize all trajs
rd.composite( tmp_path, suc_trajs[:10],  np.array([env_id])) # visualize suc trajs
img = imageio.imread(tmp_path)
utils.plt_img(img, dpi=200) # 600